In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint
import matplotlib.pyplot as plt

In [5]:
# Carregar o CSV
file_path = '/Users/usuario1/Desktop/P_ia/SP500-Prediction-Dimmy/data/sp500.csv'
data = pd.read_csv(file_path)


In [6]:
# Processamento dos dados
data['DATE'] = pd.to_datetime(data['DATE'])
data = data.set_index('DATE')
data['SP500_shifted'] = data['SP500'].shift(-1)
data.dropna(inplace=True)

In [7]:
# Criar variáveis preditoras (features) e alvo (target)
X = data[['SP500']]
y = data['SP500_shifted']

In [8]:
# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)

In [11]:
# Definir a grade de hiperparâmetros para pesquisa
param_distributions = {
    'n_estimators': randint(100, 500),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

In [12]:
# Configurar a pesquisa aleatória
random_search = RandomizedSearchCV(
    regressor, param_distributions, n_iter=100, cv=5, verbose=1, random_state=42, n_jobs=-1
)

In [13]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
160 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
143 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.1

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x303936db0>,
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x303934d10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x303934e90>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x303936930>},
                   random_state=42, verbose=1)

In [14]:
best_model = random_search.best_estimator_